In [1]:
data_path="C:\\Users\\HP\\dataset"

In [2]:
import os
import cv2

In [3]:
categories=["with_mask","without_mask"]

In [4]:
label_dict={"with_mask":0,"without_mask":1}

In [5]:
data=[]
target=[]

In [6]:
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        try:
            img=cv2.imread(img_path)
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(100,100))
            data.append(resized)
            target.append(label_dict[category])
        except:
            pass

In [7]:
import numpy as np
data=np.array(data)
data=data/255

In [8]:
data.shape

(3810, 100, 100)

In [9]:
data=np.reshape(data,(3810,100,100,1))

In [10]:
data.shape

(3810, 100, 100, 1)

In [11]:
target=np.array(target)

In [12]:
target.shape

(3810,)

In [13]:
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [14]:
new_target.shape

(3810, 2)

In [15]:
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D


In [16]:
from keras.models import Sequential

In [17]:
model=Sequential()
model.add(Conv2D(200,(3,3),input_shape=(100,100,1),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(200,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [18]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [19]:
model.fit(data,new_target,epochs=20)

Epoch 1/20
120/120 [==============================] - 166s 1s/step - loss: 0.5982 - accuracy: 0.6654
Epoch 2/20
120/120 [==============================] - 169s 1s/step - loss: 0.3700 - accuracy: 0.8315
Epoch 3/20
120/120 [==============================] - 169s 1s/step - loss: 0.3048 - accuracy: 0.8685
Epoch 4/20
120/120 [==============================] - 170s 1s/step - loss: 0.2159 - accuracy: 0.9126
Epoch 5/20
120/120 [==============================] - 188s 2s/step - loss: 0.1893 - accuracy: 0.9249
Epoch 6/20
120/120 [==============================] - 244s 2s/step - loss: 0.1500 - accuracy: 0.9383
Epoch 7/20
120/120 [==============================] - 241s 2s/step - loss: 0.1340 - accuracy: 0.9478
Epoch 8/20
120/120 [==============================] - 242s 2s/step - loss: 0.0931 - accuracy: 0.9625
Epoch 9/20
120/120 [==============================] - 242s 2s/step - loss: 0.0951 - accuracy: 0.9638
Epoch 10/20
120/120 [==============================] - 241s 2s/step - loss: 0.0593 - accura

In [71]:
import sys

In [72]:
def getClassName(target):
    if target==0: return "with_mask"
    elif target==1:return "without_mask"

In [73]:
cascPath = "C:\\Users\\HP\\haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

In [74]:
faceCascade.empty()

False

In [76]:
labels_dict={0:'with_mask',1:'without_mask'}
color_dict={0:(0,255,0),1:(0,0,255)}
font=cv2.FONT_HERSHEY_SIMPLEX
cascPath = "C:\\Users\\HP\\haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
cap=cv2.VideoCapture(0)

while True:
    success,imgOriginal=cap.read()
    img=np.asarray(imgOriginal)
    img=cv2.resize(img,(100,100))
    img=img.reshape(1,100,100,1)
    img=img/255
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        img,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    probabilities=model.predict(img)
    classIndex=model.predict_classes(img)
    probabilityValue=np.amax(probabilities)
    if probabilityValue>0.75:
            for (x, y, w, h) in faces:
                    cv2.rectangle(imgOriginal, (x, y), (x+w, y+h),color_dict[classIndex] , 2)
                    cv2.rectangle(imgOriginal, (x, y), (x+w, y+h),color_dict[classIndex] , -1)
                    cv2.putText(imgOriginal, labels_dict[classIndex], (x1, y1-10),font,0.75,(255,255,255),2)
        
                    
    cv2.imshow("Result",imgOriginal)
    if cv2.waitKey(1) and 0xFF==ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

ValueError: cannot reshape array of size 30000 into shape (1,100,100,1)